In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from os import path
import os
from urllib import parse

In [2]:
def load_json(json_fname):
    with open(json_fname,encoding="utf8") as fp:
        return json.load(fp)

In [3]:
title_key=lambda title:re.sub("\W","",title.lower(),)

# save link to file

In [182]:
{
    "title_key":{
        "title":str,
        "author":[
            str,
            str,
        ],
        "file_name":str,
        "conf":str
    }
}

{'title_key': {'title': str,
  'author': [str, str],
  'file_name': str,
  'conf': str}}

In [176]:
def parse_eccv2018():
    conf="ECCV2018"
    assert isinstance(conf,str)
    page=requests.get(f"http://openaccess.thecvf.com/{conf}.py")
    assert page.ok
    soup=BeautifulSoup(page.content,"lxml")
    dl=soup.select_one("dl")

    title_list=[a.text for a in  dl.select("dt > a")]

    authors_list=[[au.attrs["value"] for au in dd.select("form > input")]  for dd in dl.select("dt + dd")]

    dd=dl.select("dt + dd")[0]

    link_list=[ parse.urljoin("http://openaccess.thecvf.com",a.attrs["href"]) for a in dl.select("dd + dd > a")]
    file_name_list=[path.split(link)[-1] for link in link_list ]
    ll=map(lambda t,a,l,fn:dict(title=t,authors=a,link=l,conf=conf,file_name=fn) , title_list,authors_list,link_list,file_name_list)
    ll=list(ll)
    
    return ll

In [177]:
ll=[]
# ll+=parse("ECCV2016")
ll+=parse_eccv2018()

In [168]:
ll.append({
    "title": "Summarizing First-Person Videos from Third Persons' Points of Views",
    "authors": [
      "HSUAN-I HO",
      "Wei-Chen Chiu",
      "Yu-Chiang Frank Wang"
    ],
    "link": "http://openaccess.thecvf.com/content_ECCV_2018/papers/HSUAN-I_HO_Summarizing_First-Person_Videos_ECCV_2018_paper.pdf",
    "file_name":"HSUAN-I_HO_Summarizing_First-Person_Videos_ECCV_2018_paper.pdf",
    "conf": "ECCV2018"}
  )

In [169]:
with open("links.json","w") as fp:
    json.dump({"l":ll},fp,indent=2)

# 下载

In [7]:
full_text_dir="full_text"
if not path.exists(full_text_dir):
    os.mkdir(full_text_dir)

In [35]:
def download_link_d(link):
    fpath=path.join(full_text_dir,link["file_name"])
    if not path.exist(fpath):
        try:
            res=requests.get(link["link"],proxies={"http":"127.0.0.1:10800"})
            assert res.ok
            with open(fpath,"wb") as fp:
                fp.write(res.content)
        except Exception as e:
            print(f"ERROR: {link} {e}")
            raise
        else:
            print(link["title"])
            link["downloaded"]=True
    else:
        link["downloaded"]=True
    return link

In [10]:
def download(url,fpath):
    try:
        res=requests.get(url,proxies={"http":"127.0.0.1:10800"})
        assert res.ok
        with open(fpath,"wb") as fp:
            fp.write(res.content)
    except Exception as e:
        print(f"ERROR: {url} {e}")
        raise
    else:
        print(url)
#             link["downloaded"]=True

In [36]:
from multiprocessing.dummy import Pool

In [39]:
ll=load_json("links.json")["l"]

In [40]:
pool=Pool(32)
for link in ll:
    if not link.get("downloaded",False):
        pool.apply_async(download_link_d,(link,))
pool.close()
pool.join()

In [17]:
with open("links.json","w") as fp:
    json.dump({"l":ll},fp,indent=2)

In [ ]:
ll

# 存到映射

In [21]:
links_map=dict()
for link in ll:
    links_map[title_key(link["title"])]=link
with open("links_map.json","w") as fp:
    json.dump(links_map,fp,indent=2)

# 处理中国的论文

In [4]:
import shutil

In [5]:
article_list=load_json("article_list.json")["articles"]

In [11]:
# cp to target
target="eccv2018-cn"
if not path.exists(target):
    os.mkdir("eccv2018-cn")
for article in article_list:
    if article["file_name"]:
        src=path.join(full_text_dir, article["file_name"])
        dst=path.join(target,article["file_name"])
        if not path.exists(src):
            download(article["full_text_link"],src)
        shutil.copy(src,dst)
#         os.system(cmd)

http://openaccess.thecvf.com/content_ECCV_2018/papers/Yinlong_Liu_Efficient_Global_Point_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Zehao_Huang_Data-Driven_Sparse_Structure_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Xiaoqing_Yin_FishEyeRecNet_A_Multi-Context_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Lipeng_Ke_Multi-Scale_Structure-Aware_Network_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Yifan_Sun_Beyond_Part_Models_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Ming_Sun_Multi-Attention_Multi-Class_Constraint_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Yidan_Zhou_HBE_Hand_Branch_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Dongqing_Zhang_Optimized_Quantization_for_ECCV_2018_paper.pdf
http://openaccess.thecvf.com/content_ECCV_2018/papers/Gang_Zhang_Generative_Adversarial_Network_E

AssertionError: 

In [34]:
path.exists("full_text\\Shaofei_Wang_Accelerating_Dynamic_Programs_ECCV_2018_paper.pdf")

False